In [7]:
!unzip '/content/drive/MyDrive/Car damage-20221113T134214Z-001.zip'

Archive:  /content/drive/MyDrive/Car damage-20221113T134214Z-001.zip
replace Car damage/level/validation/01-minor/0011.jpeg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [8]:
#import image datagenerator library 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
#setting parameter for image data augmentation to the training data.
train_datagen = ImageDataGenerator(rescale=1./255,
                                                            shear_range=0.1,
                                                            zoom_range=0.1,
                                                             horizontal_flip=True)

In [ ]:
#image data augmentation to the testing data.
val_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory('/content/Car damage/body/training',
                                               target_size = (224, 224),
                                               batch_size = 10,
                                               class_mode = 'categorical')
test_set = val_datagen.flow_from_directory('/content/Car damage/body/validation',
                                                target_size = (224, 224),
                                               batch_size = 10,
                                               class_mode = 'categorical')  


In [ ]:
training_set = train_datagen.flow_from_directory('/content/Car damage/level/training',
                                               target_size = (224, 224),
                                               batch_size = 10,
                                               class_mode = 'categorical')
test_set = val_datagen.flow_from_directory('/content/Car damage/level/validation',
                                                target_size = (224, 224),
                                               batch_size = 10,
                                               class_mode = 'categorical')  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt  

In [ ]:
vgg=VGG16(weights='imagenet',include_top=False,input_tensor=Input(shape=(224,224,3)))
vgg1=VGG16(weights='imagenet',include_top=False,input_tensor=Input(shape=(224,224,3)))

In [ ]:
for layer in vgg.layers:
    layer.trainable=False
x=Flatten()(vgg.output)
for layer in vgg1.layers:
    layer.trainable=False
y=Flatten()(vgg1.output)

In [ ]:
prediction=Dense(3,activation='softmax')(x)
prediction1=Dense(3,activation='softmax')(y)

In [ ]:
model=Model(inputs=vgg.input,outputs=prediction)
model1=Model(inputs=vgg1.input,outputs=prediction1)

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['acc'])
model1.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['acc'])

In [ ]:
import sys
r = model.fit_generator(training_set,
                        validation_data=test_set,
                        epochs=25,
                        steps_per_epoch=979//10,
                        validation_steps=171//10)

In [ ]:
import sys
r = model1.fit_generator(training_set,
                        validation_data=test_set,
                        epochs=25,
                        steps_per_epoch=979//10,
                        validation_steps=171//10)

In [ ]:
model.save('body.h5')

In [ ]:
model1.save('level.h5')

In [ ]:
from tensorflow.keras.models import  load_model
import cv2
from skimage.transform import resize

body_model=load_model('body.h5')

In [ ]:
from tensorflow.keras.models import  load_model
import cv2
from skimage.transform import resize

level_model=load_model('level.h5')

In [ ]:
def detect(frame):
    img=cv2.resize(frame,(224,224))
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    if(np.max(img)>1):
        img=img/255.0
    img=np.array([img])
    prediction =body_model.predict(img)
    label=["front","rear","side"]
    preds=label[np.argmax(prediction)]
    return preds

In [ ]:
def detect(frame):
    img=cv2.resize(frame,(224,224))
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    if(np.max(img)>1):
        img=img/255.0
    img=np.array([img])
    prediction =level_model.predict(img)
    print(prediction)
    label=["minor","moderate","severe"]
    preds=label[np.argmax(prediction)]
    return preds

In [ ]:
import numpy as np

data= '/content/Car damage/body/training/00-front/0006.JPEG'
image= cv2.imread(data)
print(detect(image))

In [ ]:
import numpy as np

data='/content/Car damage/level/validation/01-minor/0009.jpeg'
image= cv2.imread(data)
print(detect(image))